# MATPOWER IEEE Case 9

R.P. Schulz, A.E. Turner and D.N. Ewart, "Long Term Power System Dynamics," EPRI Report 90-7-0, Palo Alto, California, 1974.

In [1]:
%load_ext autoreload
%autoreload 2

## AC power flow

In [2]:
import numpy as np

from GridCal.Engine import Bus, Load, Line, Generator, MultiCircuit
from GridCal.Engine import PowerFlowOptions, PowerFlowDriver, SolverType

Bentayga is not available
Newton native is not available


In [3]:
import os

from matpower import path_matpower

In [4]:
from GridCal.Engine.IO import parse_matpower_file

In [5]:
case_name = 'case9.m'
case_path = os.path.join(path_matpower, 'data', case_name)

grid, _ = parse_matpower_file(case_path)

In [6]:
options = PowerFlowOptions(SolverType.NR, verbose=False)
power_flow = PowerFlowDriver(grid, options)
power_flow.run()

In [7]:
print('|V|:', abs(power_flow.results.voltage))
print('|Sbranch|:', abs(power_flow.results.Sf))
print('|loading|:', abs(power_flow.results.loading) * 100)
print('err:', power_flow.results.error)
print('Conv:', power_flow.results.converged)
print('Total real losses:', sum(np.real(power_flow.results.losses)))
print('Power flow:', power_flow.results.If)

|V|: [1.04       1.025      1.025      1.02578841 1.01265435 1.03235296 1.01588261 1.0257694  0.99563089]
|Sbranch|: [ 76.57620336  30.72093467  60.96638094  85.69092124  24.38378354  76.65564245 163.25820057  87.02464825  56.13866969]
|loading|: [30.63048134 12.28837387 40.64425396 28.56364041 16.25585569 30.66225698 65.30328023 34.8098593  22.45546787]
err: 3.4213208180533117e-07
Conv: True
Total real losses: 4.641022915358031
Power flow: [ 0.68885589-0.26005666j  0.29870531-0.02161134j -0.57743512+0.17037386j  0.81790513+0.17303824j  0.23515435-0.02216017j -0.74845653+0.09587239j -1.57989878-0.1923781j   0.83736138+0.13631421j -0.38056415+0.41605047j]


In [8]:
power_flow.results.get_report_dataframe()

,Method,Converged?,Error,Elapsed (s),Iterations
0,Newton Raphson,True,3.421321e-07,0.20485,3


In [9]:
import pandas as pd

pd.options.display.float_format = '{:.3f}'.format

In [10]:
df_bus, df_branch = power_flow.results.export_all()
df_branch['Imag power (MVAr)'] = abs(df_branch['Imag power (MVAr)'])
df_branch

,Real power (MW),Imag power (MVAr),Power module (MVA),Loading(%),Losses (MW),Losses (MVAr),Losses (MVA),Tap module
0,71.641,27.046,76.576,0.306,-0.000,3.123,3.123,NaN
1,30.704,1.030,30.721,0.123,0.166,-15.513,15.514,NaN
2,-59.463,13.457,60.966,0.406,1.354,-31.531,31.561,NaN
3,85.000,10.860,85.691,0.286,0.000,4.096,4.096,NaN
4,24.183,3.119,24.384,0.163,0.088,-21.176,21.176,NaN
5,-75.905,10.704,76.656,0.307,0.475,-11.502,11.511,NaN
6,-163.000,9.178,163.258,0.653,0.000,15.832,15.832,NaN
7,86.620,8.381,87.025,0.348,2.300,-19.694,19.827,NaN
8,-40.680,38.687,56.139,0.225,0.258,-15.794,15.796,NaN


In [11]:
def round_power_close_to_zero(power):
    real_power = np.real(power)
    real_power[np.isclose(real_power,0,atol=1e-06)] = 0

    imag_power = np.imag(power)
    imag_power[np.isclose(imag_power,0,atol=1e-06)] = 0

    return real_power, imag_power

In [12]:
df_bus['Real power (MW)'], df_bus['Imag power (MVAr)'] = round_power_close_to_zero(power_flow.results.Sbus)
df_bus


,Real voltage (p.u.),Imag Voltage (p.u.),Voltage module (p.u.),Voltage angle (rad),Real power (MW),Imag power (MVAr)
0,1.040,0.000,1.040,0.000,71.641,27.046
1,1.012,0.165,1.025,0.162,163.000,6.654
2,1.022,0.083,1.025,0.081,85.000,-10.860
3,1.025,-0.040,1.026,-0.039,-0.000,0.000
4,1.011,-0.065,1.013,-0.064,-90.000,-30.000
5,1.032,0.035,1.032,0.034,0.000,0.000
6,1.016,0.013,1.016,0.013,-100.000,-35.000
7,1.024,0.067,1.026,0.065,0.000,0.000
8,0.993,-0.069,0.996,-0.070,-125.000,-50.000


## Comparison with MATPOWER result

In [13]:
from matpower import start_instance

In [14]:
m = start_instance()
mpopt = m.mpoption('verbose', 1, 'out.sys_sum', 0, 'out.bus', 0)
mpc = m.eval('case9', verbose=False)
mpc = m.runpf(mpc, mpopt)

OSQP Error!

MATPOWER Version 7.1, 08-Oct-2020 -- AC Power Flow (Newton)

Newton's method power flow (power balance, polar) converged in 4 iterations.

Converged in 0.26 seconds
|     Branch Data                                                              |
Brnch   From   To    From Bus Injection   To Bus Injection     Loss (I^2 * Z)
  #     Bus    Bus    P (MW)   Q (MVAr)   P (MW)   Q (MVAr)   P (MW)   Q (MVAr)
-----  -----  -----  --------  --------  --------  --------  --------  --------
   1      1      4     71.64     27.05    -71.64    -23.92     0.000      3.12
   2      4      5     30.70      1.03    -30.54    -16.54     0.166      0.90
   3      5      6    -59.46    -13.46     60.82    -18.07     1.354      5.90
   4      3      6     85.00    -10.86    -85.00     14.96     0.000      4.10
   5      6      7     24.18      3.12    -24.10    -24.30     0.088      0.75
   6      7      8    -75.90    -10.70     76.38     -0.80     0.475      4.03
   7      8      2   -163.00 